<a href="https://colab.research.google.com/github/utkarshsharma1/ECG_Signals/blob/master/Utkarsh_LSTM_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 62kB/s 
     |████████████████████████████████| 3.1MB 47.1MB/s 
     |████████████████████████████████| 501kB 44.1MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir

In [4]:
!pip install wfdb
import wfdb              # much needed for annotations

     |████████████████████████████████| 102kB 3.2MB/s 
     |████████████████████████████████| 163kB 9.1MB/s 
  Created wheel for wfdb: filename=wfdb-2.2.1-cp36-none-any.whl size=100368 sha256=3b2429c824afb1d7e7cb456429d66cceacd3bf5ad59407af7134291529b32887
  Stored in directory: /root/.cache/pip/wheels/bb/a9/00/0078d26b0c15b31be0001af8eb659496709c361c69641303f1
Successfully built wfdb


In [0]:
path = 'mit-bih-arrhythmia-database-1.0.0/'

In [0]:
sample_database = ['100', '101', '102', '103', '104', '105', '106', '107', '108', 
                   '109', '111', '112', '113', '114', '115', '116', '117', '118',
                   '119', '121', '122', '123', '124', '200', '201', '202', '203',
                   '205', '207', '208', '209', '210', '212', '213', '214', '215',
                   '217', '219', '220', '221', '222', '223', '228', '230', '231',
                   '232', '233', '234']

In [0]:
def load_ecg_signal(file):
    
    record = wfdb.rdrecord(file, pb_dir= 'mitdb')
    annotation = wfdb.rdann(file, 'atr', pb_dir= 'mitdb')
    
    #This gives signal
    p_signal = record.p_signal
    
    assert record.fs == 360, 'Sample frequency is not 360'
    
    #This gives symbols & annotation index
    atr_symbol = annotation.symbol
    atr_sample = annotation.sample
    
    return p_signal, atr_symbol, atr_sample

In [0]:
abnormal = ['L', 'R', 'V', '/', 'A', 'f', 'F', 'j', 'a', 'E', 'J', 'e', 'S']

In [0]:
def make_dataset(sample_database, no_of_sec, fs):
    
    #No of columns
    num_cols = 2 * no_of_sec * fs
    X_all = np.zeros((1, num_cols))
    Y_all = np.zeros((1,1))
    symbol_all = []
    
    max_rows = []
    
    for i in sample_database:
        #file = path + i
        print(i)
        p_signal, atr_symbol, atr_sample = load_ecg_signal(i)
        
        #Take out first signal as there are 2 signals
        p_signal = p_signal[:,0]
        
        #Make a dataframe to exclude irrevalent atr_symbol
        #atr_sample is index of annotation
        df_annotation = pd.DataFrame({'atr_symbol': atr_symbol,'atr_sample': atr_sample})
        df_annotation = df_annotation.loc[df_annotation.atr_symbol.isin(abnormal + ['N'])]
        
        #Gives no of accepted pulses
        no_of_rows = len(df_annotation)
        
        #creates matrix accordingly
        X = np.zeros((no_of_rows, num_cols))
        Y = np.zeros((no_of_rows, 1))
        
        #keep track of rows
        row_number = 0
        
        #Now iterate through the dataframe
        for atr_sample, atr_symbol in zip(df_annotation.atr_sample.values, df_annotation.atr_symbol.values):
            
            #Took min and max value in order to tackle the 2 extreme end cases
            left = max([0, (atr_sample - no_of_sec * fs)])
            right = min([len(p_signal), (atr_sample + no_of_sec * fs)])
            x = p_signal[left: right]
            
            if len(x) == num_cols:
            
                #This will store all the values of p_signal from 'left' to 'right'
                X[row_number, :] = x
                
                #This line will store 0 if its normal & 1 if its abnormal in Y
                Y[row_number, :] = int(atr_symbol in abnormal)
                
                #A list consisting of all symbols
                symbol_all.append(atr_symbol)
                row_number += 1
                
        #Reduces rows of X and Y as we remove some datas whose len(x) != num_cols
        X = X[: row_number, :]
        Y = Y[: row_number, :]
                
        #For checking dimensions       
        max_rows.append(row_number)
                
        #append X and Y from previous iteration with new data
        X_all = np.append(X_all, X, axis = 0)
        Y_all = np.append(Y_all, Y, axis = 0)
     
    #Removes first column as it contains only zeros when we initialized it
    X_all = X_all[1:, :]
    Y_all = Y_all[1:, :]
            
    
    assert np.sum(max_rows) == X_all.shape[0], 'No of rows messed 1'
    assert Y_all.shape[0] == X_all.shape[0], 'No of rows messed 2' 
    assert Y_all.shape[0] == len(symbol_all), 'No of rows messed 3'
            
    return X_all, Y_all, symbol_all

In [0]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

def calc_prevalence(Y_actual):
  return sum(Y_actual)/ len(Y_actual)

def calc_specificity(Y_actual, Y_pred, thresh):
  return sum((Y_pred < thresh) & (Y_actual == 0))/ sum(Y_actual == 0)

def print_report(Y_actual, Y_pred, thresh):

  auc = roc_auc_score(Y_actual, Y_pred)
  accuracy = accuracy_score(Y_actual, (Y_pred > thresh))
  recall = recall_score(Y_actual, (Y_pred > thresh))
  precision = precision_score(Y_actual, (Y_pred > thresh))
  specificity = calc_specificity(Y_actual, Y_pred, thresh)
  prevalence = calc_prevalence(Y_actual)
  print('AUC: %.3f' %auc)
  print('Accuracy: %.3f' %accuracy)
  print('Recall: %.3f' %recall)
  print('Precision: %.3f' %precision)
  print('Specificity: %.3f' %specificity)
  print('Prevalence: %.3f' %prevalence)
  print('')
  return auc, accuracy, recall, precision, specificity


In [11]:
#Technically the same patient can show up in both the training and validation sets. 
#This means that we may have accidentally leaked information across the datasets. 
#We can test this idea by splitting on patients instead of samples.
import random
random.seed(27)

pts_train = random.sample(sample_database, 36)
pts_valid = [pt for pt in sample_database if pt not in pts_train]
print(len(pts_train), len(pts_valid))

36 12


In [12]:
#Lets keep no_of_sec as -+ of 3
no_of_sec = 3
#sampling frequency is 360
fs = 360

X_train, Y_train, Sym_train = make_dataset(pts_train, no_of_sec, fs)
X_valid, Y_valid, Sym_valid = make_dataset(pts_valid, no_of_sec, fs)
print(X_train.shape, Y_train.shape, len(Sym_train))
print(X_valid.shape, Y_valid.shape, len(Sym_valid))

223
209
231
118
119
113
104
234
117
214
123
221
200
202
112
116
212
210
203
102
215
122
230
208
207
121
114
219
101
115
213
100
233
222
205
109
103
105
106
107
108
111
124
201
217
220
228
232
(84802, 2160) (84802, 1) 84802
(24297, 2160) (24297, 1) 24297


In [0]:
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [17]:
X_train_cnn = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_valid_cnn = np.reshape(X_valid, (X_valid.shape[0], X_valid.shape[1], 1))

print(X_train_cnn.shape)
print(X_valid_cnn.shape)

(84802, 2160, 1)
(24297, 2160, 1)


In [0]:
model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape = (X_train_cnn.shape[1], X_train_cnn.shape[2]))))
model.add(Dropout(rate = 0.25))
model.add(Dense(1, activation = 'sigmoid'))

In [0]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [0]:
model.fit(X_train_cnn, Y_train, batch_size = 32, epochs = 2, verbose = 1)

Train on 84802 samples
Epoch 1/2
10976/84802 [==>...........................] - ETA: 1:08:34 - loss: 0.5837 - accuracy: 0.7291

In [0]:
Y_train_preds_lstm = model.predict_proba(X_train_cnn, verbose = 1)
Y_valid_preds_lstm = model.predict_proba(X_valid_cnn, verbose = 1)

In [0]:
#It calculates the ratio of number of values of Y_train = 0 to size of Y_train
thresh = (sum(Y_train)/ len(Y_train))[0]
thresh

In [0]:
print('Train');
print_report(Y_train, Y_train_preds_cnn, thresh)
print('Valid');
print_report(Y_valid, Y_valid_preds_cnn, thresh)